In [69]:
import time
import os

import pandas as pd
import pickle

import requests
from pymongo import MongoClient

import spotipy

### Connecting to MongoDB

In [7]:
client = MongoClient()

In [8]:
db = client.spotify

In [9]:
db.list_collection_names()

['tracks', 'artists', 'albums']

In [12]:
cursor = db.tracks.find({}, 
    {'_id':0, 'album_uri':1, 'artist_id':1, 'artist_name':1, 'artist_uri':1, 'duration_ms':1, 'explicit':1, 'id':1, 'name':1, 'track_number':1, 'uri':1, 'song_spotify_page':1})
df = pd.DataFrame(list(cursor))
df.head()

,album_uri,artist_id,artist_name,artist_uri,duration_ms,explicit,id,name,song_spotify_page,track_number,uri
0,spotify:album:2h6MSR1rWemHOtmurgRq3T,7kAKqNxPBkfjgdHzUIdBtI,Suburban Kids With Biblical Names,spotify:artist:7kAKqNxPBkfjgdHzUIdBtI,138626,False,4okEZakOVppAtP4Dawd52x,Marry Me,https://open.spotify.com/track/4okEZakOVppAtP4...,1,spotify:track:4okEZakOVppAtP4Dawd52x
1,spotify:album:2h6MSR1rWemHOtmurgRq3T,7kAKqNxPBkfjgdHzUIdBtI,Suburban Kids With Biblical Names,spotify:artist:7kAKqNxPBkfjgdHzUIdBtI,187106,False,2LV6sB5zTsu0R5r5kWohlD,Loop Duplicate My Heart,https://open.spotify.com/track/2LV6sB5zTsu0R5r...,2,spotify:track:2LV6sB5zTsu0R5r5kWohlD
2,spotify:album:2h6MSR1rWemHOtmurgRq3T,7kAKqNxPBkfjgdHzUIdBtI,Suburban Kids With Biblical Names,spotify:artist:7kAKqNxPBkfjgdHzUIdBtI,192000,False,2RPcqvy4bdznms2eHU6vnE,A Couple Of Instruments,https://open.spotify.com/track/2RPcqvy4bdznms2...,3,spotify:track:2RPcqvy4bdznms2eHU6vnE
3,spotify:album:2h6MSR1rWemHOtmurgRq3T,7kAKqNxPBkfjgdHzUIdBtI,Suburban Kids With Biblical Names,spotify:artist:7kAKqNxPBkfjgdHzUIdBtI,176026,False,53uzYuKe433aXBjzBiuvqe,Parakit,https://open.spotify.com/track/53uzYuKe433aXBj...,4,spotify:track:53uzYuKe433aXBjzBiuvqe
4,spotify:album:2h6MSR1rWemHOtmurgRq3T,7kAKqNxPBkfjgdHzUIdBtI,Suburban Kids With Biblical Names,spotify:artist:7kAKqNxPBkfjgdHzUIdBtI,198013,False,7p0pJgizlHS5msrgDPU6li,Trees And Squirrels,https://open.spotify.com/track/7p0pJgizlHS5msr...,5,spotify:track:7p0pJgizlHS5msrgDPU6li


## Removing non-queried artists

In [72]:
returned_artists = df.artist_name.unique().tolist()

In [70]:
pickle_in = open('Data/indie_pop_artists','rb')
queried_artists = pickle.load(pickle_in)
pickle_in.close()

In [73]:
print(len(queried_artists))
print(len(returned_artists))
print(len(set(queried_artists).intersection(set(returned_artists))))

716
714
539


In [74]:
artists_df = pd.DataFrame(list(set(queried_artists).intersection(set(returned_artists))))
artists_df.columns = ['artist_name']
artists_df.head()

,artist_name
0,The Silly Pillows
1,Urban Cone
2,Tegan and Sara
3,Rose Melberg
4,Geographer


In [75]:
df = df.merge(artists_df,on='artist_name')
df.head()

,album_uri,artist_id,artist_name,artist_uri,duration_ms,explicit,id,name,song_spotify_page,track_number,uri
0,spotify:album:2h6MSR1rWemHOtmurgRq3T,7kAKqNxPBkfjgdHzUIdBtI,Suburban Kids With Biblical Names,spotify:artist:7kAKqNxPBkfjgdHzUIdBtI,138626,False,4okEZakOVppAtP4Dawd52x,Marry Me,https://open.spotify.com/track/4okEZakOVppAtP4...,1,spotify:track:4okEZakOVppAtP4Dawd52x
1,spotify:album:2h6MSR1rWemHOtmurgRq3T,7kAKqNxPBkfjgdHzUIdBtI,Suburban Kids With Biblical Names,spotify:artist:7kAKqNxPBkfjgdHzUIdBtI,187106,False,2LV6sB5zTsu0R5r5kWohlD,Loop Duplicate My Heart,https://open.spotify.com/track/2LV6sB5zTsu0R5r...,2,spotify:track:2LV6sB5zTsu0R5r5kWohlD
2,spotify:album:2h6MSR1rWemHOtmurgRq3T,7kAKqNxPBkfjgdHzUIdBtI,Suburban Kids With Biblical Names,spotify:artist:7kAKqNxPBkfjgdHzUIdBtI,192000,False,2RPcqvy4bdznms2eHU6vnE,A Couple Of Instruments,https://open.spotify.com/track/2RPcqvy4bdznms2...,3,spotify:track:2RPcqvy4bdznms2eHU6vnE
3,spotify:album:2h6MSR1rWemHOtmurgRq3T,7kAKqNxPBkfjgdHzUIdBtI,Suburban Kids With Biblical Names,spotify:artist:7kAKqNxPBkfjgdHzUIdBtI,176026,False,53uzYuKe433aXBjzBiuvqe,Parakit,https://open.spotify.com/track/53uzYuKe433aXBj...,4,spotify:track:53uzYuKe433aXBjzBiuvqe
4,spotify:album:2h6MSR1rWemHOtmurgRq3T,7kAKqNxPBkfjgdHzUIdBtI,Suburban Kids With Biblical Names,spotify:artist:7kAKqNxPBkfjgdHzUIdBtI,198013,False,7p0pJgizlHS5msrgDPU6li,Trees And Squirrels,https://open.spotify.com/track/7p0pJgizlHS5msr...,5,spotify:track:7p0pJgizlHS5msrgDPU6li


In [76]:
df.shape

(35538, 11)